In [5]:
import numpy as np
import json

In [6]:
with open('sudoku_01.json') as f:
    d1 = json.load(f)
d1

[[5, 3, 0, 0, 7, 0, 0, 0, 0],
 [6, 0, 0, 1, 9, 5, 0, 0, 0],
 [0, 9, 8, 0, 0, 0, 0, 6, 0],
 [8, 0, 0, 0, 6, 0, 0, 0, 3],
 [4, 0, 0, 8, 0, 3, 0, 0, 1],
 [7, 0, 0, 0, 2, 0, 0, 0, 6],
 [0, 6, 0, 0, 0, 0, 2, 8, 0],
 [0, 0, 0, 4, 1, 9, 0, 0, 5],
 [0, 0, 0, 0, 8, 0, 0, 7, 9]]

In [ ]:
class sudoku:
    def __init__(self, dim: int):
        self.dim = dim
        self.tau = self.create_tau(dim) 
        
    @classmethod
    def create_tau(dim):
        block = np.zeros((dim,dim))
        n = dim*dim
        tau = {}
        for i in range(n):
            for j in range(n):
                tau[f'{i}{j}'] = []
                tau[f'{i}{j}'].exdend([f'{i}{k}' for k in range(n) if k!=i])
                
    

In [2]:
block = np.array(if i!=0 :
                 [[f'{i}{j}' for j in range(3)] for i in range(3) ])
                
block

array([['00', '01', '02'],
       ['10', '11', '12'],
       ['20', '21', '22']], dtype='<U2')

In [4]:
np.zeros((3,3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [8]:
np.array([if i!=0 :
          [f'{i}{j}' for j in range(3)] for i in range(3) ])

SyntaxError: invalid syntax (<ipython-input-8-f1a18cbc9be0>, line 1)

In [9]:
np.append([], [1,2])

array([1., 2.])

In [17]:
a=[1,2,2]
a.remove(2)
a

[1, 2]

In [21]:
n = 9
tau = {}
for i in range(n):
    for j in range(n):
        tau[f'{i}{j}'] = []
        tau[f'{i}{j}'].extend([f'{i}{k}' for k in range(n) if k!=j])
        tau[f'{i}{j}'].extend([f'{k}{j}' for k in range(n) if k!=i])
        

tau['01']

['00', '02', '03', '04', '05', '06', '07', '08']